<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/testingserver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !pip install simpletransformers
# !pip install torch
!pip install pythainlp
!pip install python-crfsuite

In [ ]:
!rm -rf '/content/ngrok-stable-linux-amd64.tgz'
!rm -rf '/content/ngrok'

!pip install flask --quiet
!pip install flask-ngrok --quiet

# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

# extract the downloaded file using the following command 
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

# paste your AuthToken here and execute this command
!./ngrok authtoken 2Q8ZedCqtQIP2l7e7KVREa2MMvc_2qRULQx4QRLYG97GMTyaZ

In [ ]:
# import logging
# import torch
# from simpletransformers.t5 import T5Model, T5Args

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# model_args = T5Args()
# model_args.max_length = 100
# model_args.length_penalty = 1
# model_args.num_beams = 10

# device = torch.cuda.is_available()
# path = '/content/gdrive/MyDrive/model/mt5-4bs-4e-5lr'

# model = T5Model("mt5", path, args=model_args, use_cuda=device)

In [ ]:
import re
import pycrfsuite
from pythainlp import sent_tokenize

def split_sentence_TH(TH_predict):
  TH_predict = sent_tokenize(TH_predict)
  for i in range(len(TH_predict)):
    # remove space in back
    TH_predict[i] = TH_predict[i].rstrip()

    # debug for engine fail example ['750ml', '1500ml']
    # move to append previous sentences
    # and replace ''
    if re.search('[0-9]', TH_predict[i][0]):
      j = i
      while True:
        if TH_predict[j-1] != '':
          TH_predict[j-1] += ' ' + TH_predict[i]
          break
      j -= 1
      TH_predict[i] = ''

  # remove '' in list
  while '' in TH_predict:
    TH_predict.remove('')
  print('[split sentence completed]')
  print(TH_predict)
  return TH_predict

def split_sentence_EN(EN_predict):
  EN_predict = EN_predict.split('. ')
  for i in range(len(EN_predict)):
    if EN_predict[i][-1] != '.':
      EN_predict[i] += '.'
  return EN_predict
  
# def predict(text):
#   predictions = model.predict(text)
#   print('[split sentence completed]')
#   print(predictions)
#   return predictions

In [19]:
!pip install -U flask-cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import logging
# import Flask from flask module
from flask import Flask, request, jsonify
from flask_cors import CORS
# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok
  
app = Flask(__name__) #app name
logging.basicConfig(level=logging.INFO)
logging.getLogger('flask_cors').level = logging.DEBUG
CORS(app)
run_with_ngrok(app)
  
@app.route("/", methods=["GET", "POST"])
def index():
  if request.method == "POST":
    json_data = request.get_json()
    if not json_data:
      return jsonify({"error": "no data"})
    try:
      if json_data["task"] == "th-en":
        prediction = split_sentence_TH(json_data["text"])
      elif json_data["task"] == "en-th":
        prediction = split_sentence_TH(json_data["text"])
      # prediction = predict(prediction)
      # ทดสอบส่งกลับ
      data_predicted = {"prediction": prediction}
      return jsonify(data_predicted)
    except Exception as e:
      return jsonify({"error": str(e)})
  return "OK"
  
if __name__ == "__main__":
  app.run()

DEBUG:flask_cors.extension:Configuring CORS with resources: {'/*': {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}}


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://07c6-35-221-221-44.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


DEBUG:flask_cors.extension:Request to '/' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', '*')])
INFO:werkzeug:127.0.0.1 - - [22/May/2023 14:51:24] "GET / HTTP/1.1" 200 -
DEBUG:flask_cors.extension:Request to '/' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Set

In [ ]:
# code from stackoverflow
from flask import Flask, request, jsonify, make_response

flask_app = Flask(__name__)

@flask_app.route("/api/orders", methods=["POST", "OPTIONS"])
def api_create_order():
    if request.method == "OPTIONS": # CORS preflight
        return _build_cors_preflight_response()
    elif request.method == "POST": # The actual request following the preflight
        # น่าจะต้องใส่โคดตรงนี้มั้ง
        return _corsify_actual_response(jsonify(...))
    else:
        raise RuntimeError("Weird - don't know how to handle method {}".format(request.method))

def _build_cors_preflight_response():
    response = make_response()
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add('Access-Control-Allow-Headers', "*")
    response.headers.add('Access-Control-Allow-Methods', "*")
    return response

def _corsify_actual_response(response):
    response.headers.add("Access-Control-Allow-Origin", "*")
    return response